# Introducing nbdev: A Notebook-Driven Development Platform

## TO DO IN THIS NOTEBOOK
- [x] Restructure as a much tighter review of nbdev and how it can help in the development of a package to be used for our dashboard project
- [x] Be sure to mention the challenges of using nbdev, notably the need to be careful about what you export to packages (do we keep discussion of literate programming in here?)
- [x] Add hyperlinks to the nbdev documentation and other resources
- [ ] REMOVE THE ANSWERS BELOW ONCE FULLY TESTED (save to answers?)

## What is nbdev?

- [nbdev](https://nbdev.fast.ai) - a notebook-driven development platform. Simply write notebooks with lightweight markup and get high-quality documentation, tests, continuous integration, and packaging.

- [modular nbdev](https://nbdev.fast.ai/tutorials/modular_nbdev.html) - makes use of only the most basic features of nbdev. It will allow us take any standalone notebook and turn it into a Python script or module. Modular nbdev will allow us to bridge the gap between messy, exploratory tutorial notebooks and clean, well-organized Python modules that we can import from. 

The following introduction to "modular nbdev" is a much simpler than the full [nbdev tutorial](https://nbdev.fast.ai/tutorials/tutorial.html) because we won't create a continuous integration Python package that can be uploaded to PyPi and/or Anaconda. We will simply use notebooks to create clean Python modules that we can use to build our dashboards and web apps. From here on out, when we mention "nbdev", we are really referring to "modular nbdev."

### End Goal

The end goal of this tutorial notebook is to understand how we can use nbdev to create a `dashboard` Python module that contains all the files we need for our dashboard.

# nbdev Exports Tutorial

### Why?
While Jupyter notebooks can be exported to a Python file, nbdev allows us to export only some notebook cells, those that we choose, into a Python module. The real benefit of this approach lies in the ability to allow your notebook to tell the story of how your code came into its final form. This programming paradigm where the code provides the documentation for decisions its design is called [literate programming](https://en.wikipedia.org/wiki/Literate_programming).

Instead of cleaning up your notebooks when you have achieved your coding goal, you can leave in all the valuable information about its history including (1) showing intermediate steps for clarity and testing and
(2) documenting failed attempts and design decisions. This can be very helpful for another developer (or our future-selves) who may be trying to understand our code.

### How? Creating the first file of our `dashboard` module

Now we'll use nbdev to create a Python module - `dashboard` - that will contain the Python code needed to build our dashboard in several files.  Each file will be built using a separate Jupyter notebook.

#### `#| default_exp`: Defining the target file for export

The first thing we need to do is tell nbdev what file we intend to export selected cells to.

The `default_exp` directive is required of all notebooks used by nbdev. Notice that nbdev directives all start with "#|". This requires that we add a default export directive to our notebook. It tells nbdev what file to export to.

We are intending to create a Python package called `dashboard` that will contain all the files we need for our dashboard. By convention, it will be created in a directory called `dashboard`.  We define that directory below, but here we define the files within that directory.

We are going to create a few line and cell magics that we will use through the tutorial. We want these magic functions to be accessible anywhere. Therefore, if we want them to go in `__init__.py` the default export for this notebook needs to be `__init__`.

In [ ]:
# TODO: Add default export directive to place exported cells in the `__init__.py`
# file of this package.

 You can also give it a “dotted module names” as is done with Python packages. For example, a default export to `some.module` will create a file called `some/module.py` within the package directory.

#### `#| export`: Marking a cell for export

This directive tells nbdev that we want to export the code in this notebook cell to the module specified by `default_exp`. Notice that we need to export everything that we want to end up in the Python file, so we can't forget to export our imports.

- `#| export` Gotchas

  - All nbdev directives (those starting with `#| ` must be in the first line of the cell.

  - When performing an export, the cell can only contain imports *or* any other Python commands other than imports, but not both.  This is due to the way nbdev parses the cells in order to build a representation of the documentation.

  - The great benefit of using nbdev for development can also be the most challenging part of using nbdev - making sure we have not forgotten to export all the right cells to our resulting Python file. If you ever get a confusing or unexpected error on importing a module created with nbdev, it's very likely you forgot to export one of your code cells.

In [ ]:
# TODO: add the missing export directive to export this cell to the dashboard module
from IPython import get_ipython
from IPython.core.magic import register_line_magic, register_cell_magic
import re
import sys
import os

### Defining the `%exception` cell magic

Below is the code that defines the cell magic function called `exception`. 

**Motivation**: Often in tutorials, we want to demonstrate an intentional error. The default behavior of a Jupyter Notebook is to print the full traceback, which can be quite lengthy and distracting. In addition, if the participant wants to "Restart and Run All Cells" due to some problem they encountered, the execution will get hung up where the error is. In our case, when running a cell with an error for demonstrative purposes, we want to display the error, but we don't want to display the traceback or prevent the execution of the next cell. This cell magic achieves this desired behavior.

We will define the `exception` cell magic function below and mark this cell for export to the `dashboard` module.

In [ ]:
#| export
@register_cell_magic('exception')
def exception(line, cell):
    ip = get_ipython()
    try:
        exec(cell, None, ip.user_ns)
    except Exception as e:
        etype, value, tb = sys.exc_info()
        value.__cause__ = None  # suppress chained exceptions
        ip._showtraceback(etype, value, ip.InteractiveTB.get_exception_only(etype, value))

As noted earlier, a benefit of literate programming is that we can test out this function in-line and retain those tests. We don't have to delete the exploratory or explanatory code.

The next cell shows what happens when we use the cell magic. Notice that there isn't an export directive at the top, so this breaking error isn't exported to our Python module.

In [ ]:
%%exception

5 / 0

You can compare that output to the output below, which includes a traceback by default because the cell magic wasn't used.

In [ ]:
5 / 0

Isn't nbdev great! Instead of playing with toy examples, we can build a sophisticated, clean Python application from a series of tutorial-style notebooks. All the Python modules and packages in this tutorial were produced from the tutorial notebooks.

### Defining the `%answer` line magic

**Motivation**: The code below allows us to import code from the nbdev-generated Python files like the one we loaded above. You can think of this function working like the `%load` line magic, which pulls in code from an external file. But instead of loading the entire file, this function loads only the Python code that came from an exported notebook cell.

Let's define the `answer` line magic function below and mark this cell for export to the `dashboard` module.

In [ ]:
#| export
@register_line_magic('answer')
def answer(inputs):
    '''
    This is a cell magic called answer that allows tutorial goers to import the correct answer from the key. 
    '''
    words = []
    for word in inputs.split(' '):
        if not word.startswith('#') and len(word) != 0:
            words.append(word)
        else:
            break

    flag = False
    if len(words) == 2:
        if words[1] == '-e':
            flag = True
        else:
            filepath = words[0]
            cell_number = int(words[1])

            with open(filepath, 'r') as file:
                lines = file.readlines()

            pattern = r'# %%\s+(.+)\s+(\d+)'
            start_line = None
            end_line = None

            for i, line in enumerate(lines):
                if re.match(pattern, line):
                    match = re.search(pattern, line)
                    if match and int(match.group(2)) == cell_number:
                        start_line = i + 1
                        break
            if start_line is not None:
                for i in range(start_line, len(lines)):
                    if re.match(pattern, lines[i]):
                        end_line = i
                        break
                else:
                    end_line = len(lines)

            if start_line is not None and end_line is not None:
                code_chunk = f"#| export\n# %answer {inputs}\n\n" + ''.join(lines[start_line:end_line])
                code_chunk = code_chunk.rstrip("\n")
                get_ipython().set_next_input(code_chunk, replace=True)
            else:
                raise Exception(f"Cell number {cell_number} not found in the Python file.")

    if len(words) == 1 or words[1] == '-e':
        filepath = words[0]
        with open(filepath, 'r') as file:
            lines = file.readlines()
        code_chunk = ''.join(lines[:])
        if flag:
            code_chunk = f"# %%export {filepath}\n\n" + code_chunk
        else: 
            code_chunk = f"# %answer {filepath}\n\n" + code_chunk
        get_ipython().set_next_input(code_chunk, replace=True)

    with open(filepath, 'r') as file:
        lines = file.readlines()

#### Using the `%answer` cell magic 

If later during this tutorial you discover you have trouble coming up with an answer on your own, you can always import it from the answer key! These tutorial notebooks will provide the `%answer` cell magics (usually commented out) so you can use if you get stuck. These `%answer` cell magics require parameters which will be provided for you.**

##### Pulling the answer from a non-exported file

Sometimes we will want to check an answer from a cell that isn't a module exported by nbdev. In this case, we will provide the file path only.

##### Pulling the answer from an exported module

The following cell shows how to pull in an answer to a cell that starts with `#| export`. The path to the answer key files is the same as the default, but is preceded by the `key` directory. The example in the next cell loads the code from the 6th cell of this notebook, which is the cell we export with the import statements.

In [ ]:
# Uncomment the following line to test the answer magic
# %answer key/dashboard/__init__.py 8

## `nb_export`: Exporting the notebook to a Python module

Finally, we can use the `nb_export` function to export the cells we selected into a Python file. The second parameter tells nbdev which package to export the module to. Let's go ahead and export this notebook to the `dashboard` package.

Note that the `nb_export` function reads the notebook from disk before creating the package directory, so you will need to save your notebook before running the export!

> **IMPORTANT**: Do not forget to save before exporting the notebook, or your most recent changes will not make it into `__init__.py` file in the `dashboard` directory. It is also a good habit to "Restart Kernel and Clear All Outputs" and then save your notebook at the same time as you export. That way, when you get around to committing your code, your `git diff` isn't filled up with a bunch of serial numbers and other unintelligible things.

In [ ]:
from nbdev.export import nb_export

# The nb_export command requires the notebook name and the target library name
nb_export('02a_nbdev.ipynb', 'dashboard')

Usually, we will put `nb_export` at the end of our file for convenience, but we will continue on from here.

## Using the exported `dashboard` module

Another way to verify that this worked is to import the function we just wrote to the Python file. Can you guess what that import statement would look like?

In [11]:
# %answer 02a/01.py

# import ...

So for most modules, the import statement would look like `dashboard.__init__` but the `__init__` file is special, and will be imported any time we make an import from `dashboard` package.

Great job! Now we know the basic idea of what nbdev does. Go ahead and use the `%load` line magic inspect the exported file below.

In [14]:
# %answer key/02a/02.py

# %load pkg/module.py

There are a few things to notice here:

1. The first thing to notice is that we get a warning about this file being autogenerated. That is, if we make changes to `dashboard/__init__.py`, those changes will be overwritten the next time we run `nb_export`.

2. The second thing to notice is that there is a handy comment to tell us where the code came from. `# %% ../02a_nbdev.ipynb 10` tells us that the code came from the 10th code cell.

### The End

We will not go over the code below (although it is exported to the `dashboard` module, we won't explicitly use it).

  Proceed to the next notebook.
___

In [ ]:
#| export
@register_cell_magic('export')
def export(line, cell=None):
    line_args = line.split()
    export_filepath = None

    if len(line_args):
        export_filepath = line_args[0]
        directory = os.path.dirname(export_filepath)
        os.makedirs(directory, exist_ok=True)
        with open(export_filepath, 'w') as file:
            file.write(cell)
        print(f"exported to {export_filepath}")
            
    processed_lines = []
    for line in cell.split('\n'):
        comment_match = re.search(r'#', line)
        if comment_match:
            line = line[comment_match.start():]
            processed_lines.append(line)

    processed_cell = '\n'.join(processed_lines)
    if len(line_args):
        processed_cell = '# %answer ' + line_args[0] + '\n\n' + processed_cell
    get_ipython().set_next_input(processed_cell, replace=True)